In [1]:
import pandas as pd 
import requests
import yfinance as yf 
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
# get list of S&P500 companies

URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
r = requests.get(URL)
if r.ok:
    print(f"requests SUCCESS: '{URL}'")
else:
    print(f"requests FAIL: '{URL}'")
    
soup = BeautifulSoup(r.content, features='lxml')

table = soup.find(name='table', attrs={'id':"constituents"})
sp500_comps = []
for row in table.find_all('tr')[1:]:
    ticker = row.find(name='a', attrs={"rel":"nofollow", "class":"external text"})
    sp500_comps.append(ticker.get_text())

requests SUCCESS: 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'


In [3]:
def get_close_prices(tickers, max_attempts=5, current_attempt=0, prices=None):
    # recursion, retry until all tickers are downloaded
    # or maximum attempt is reached.
    
    if len(tickers) == 0:
        print("All data is downloaded!")
        return prices
    
    if current_attempt >= max_attempts:
        print('Max attempt is reached!')
        print(f'Fail tickers:{tickers}')
        return prices
    
    if prices is None:
        prices = pd.DataFrame()
    
    current_attempt += 1
    new_tickers = tickers.copy()
    for ticker in tqdm(tickers, leave=False, desc=f"attempt{current_attempt}"):
        if '.' in ticker: 
            yf_ticker = ticker.replace('.', '-')
        else:
            yf_ticker = ticker
            
        price = yf.download(yf_ticker, period='5y', interval='1d', progress=False, auto_adjust=True).Close
        if not price.empty:
            new_tickers.remove(ticker)
            prices = prices.join(price.rename(ticker), how='outer')
        
    return get_close_prices(tickers=new_tickers, current_attempt=current_attempt, prices=prices)

In [4]:
price_df = get_close_prices(sp500_comps)

attempt1:   0%|          | 0/505 [00:00<?, ?it/s]

All data is downloaded!


In [5]:
price_df.dropna(axis=1, thresh=len(price_df)-10, inplace=True)
price_df.ffill(inplace=True)
price_df.bfill(inplace=True)
print(f"shape: {price_df.shape}")
print(f"NA values: {price_df.isna().sum().sum()}")

shape: (1262, 494)
NA values: 0


In [6]:
price_df

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-03-16,164.992310,48.105995,41.482277,52.188644,123.970001,115.456360,47.647846,38.498363,127.010002,94.584946,...,21.149691,121.000206,102.611176,37.994030,46.323086,58.561855,90.750000,113.558990,38.701851,52.387356
2017-03-17,164.992310,48.105995,41.482277,52.188644,123.970001,115.456360,47.647846,38.498363,127.010002,94.584946,...,21.149691,121.000206,102.611176,37.994030,46.323086,58.561855,90.750000,113.558990,38.701851,52.387356
2017-03-20,165.549866,48.078339,41.289509,52.276047,123.589996,114.691086,47.482979,38.610275,126.309998,93.991028,...,21.275536,121.214340,103.961433,37.768753,46.012131,58.745235,90.070000,113.540321,38.197727,52.658989
2017-03-21,164.795029,46.990253,41.124275,51.918518,122.269997,114.921570,46.765305,38.696362,125.070000,92.794212,...,20.994228,120.283287,104.063881,38.435928,45.286579,58.433502,86.150002,112.914978,36.119370,51.795582
2017-03-22,164.872238,47.027138,41.133453,52.053585,123.470001,116.618141,47.211426,39.264538,126.209999,93.298126,...,21.053453,120.432259,105.553818,38.565891,46.436161,58.057568,86.860001,113.288300,36.022083,52.261242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-10,143.929993,66.610001,117.000000,149.169998,305.619995,309.519989,80.800003,84.309998,438.950012,209.429993,...,32.490002,224.520004,76.529999,69.673958,83.230003,115.820000,388.790009,119.339996,64.519997,189.220001
2022-03-11,140.960007,65.720001,114.019997,149.059998,286.790009,311.579987,80.309998,83.599998,416.380005,207.270004,...,31.900000,220.729996,72.320000,69.882500,83.620003,116.260002,386.070007,118.820000,64.209999,185.289993
2022-03-14,143.479996,65.879997,115.519997,152.100006,277.549988,307.679993,79.550003,82.570000,411.500000,207.000000,...,31.070000,220.100006,66.879997,69.510002,83.989998,113.419998,380.089996,117.690002,65.000000,184.660004


In [7]:
price_df.to_csv('yfdata/sp500_comps.csv')